In [3]:
%run functions_NoiseStudy.py
%matplotlib 

# global variables
fs = 13
type_plot = ['png', 'svg']

Using matplotlib backend: MacOSX


In [4]:
# actual evaluation time
now = datetime.datetime.now()
today = now.strftime("%Y%m%d")

# output folder
save_dir_plots = 'plots/' + today + '_measurement'
if not os.path.exists(save_dir_plots):
    os.makedirs(save_dir_plots)
    
save_dir_res = 'Results/' + today + '_measurement'
if not os.path.exists(save_dir_res):
    os.makedirs(save_dir_res)
    
fig_Rpen = None

In [5]:
# Paths must be updated according to your directory !!!
# depth profile microsensor
path_ = '/Volumes/HIS-CAMEL/04measurementData/20201127_Noise-vs-resolution-paper/'
file_ms = path_ + 'Klaus_Optode_noise_study_26-11-2020/O2gradient_experiement/profiles.txt'

# calibration data 
file_calib = 'Results/20210531_calibration/20210531-1455_calibration_1RoI.hdf5'

# measurement data - images of the optode.
file_meas = path_ + 'Klaus_Optode_noise_study_26-11-2020/O2gradient_experiement/'

# image cropping to the region of interest (RoI) that should be analyzed
RoI_op = [[(730, 200), (730, 1250), (1290, 1250), (1290, 200)],   # optode1
          [(1560, 200), (1560, 1250), (2100, 1250), (2100, 200)]] # optode2

# Image resolution - determine conversion factor px -> mm 
px2mm, dpi = image_resolution(px=840.6646, dist_mm=30., inch=1.1811)
print('Image resolution: 1mm equals: {:.2f}px, i.e. {:.0f}dpi'.format(px2mm, dpi))

# ................................................................................
# load depth profile(s) of the microsensir for validation of the depth profile
dic_micro = read_microsensor(file_ms=file_ms, encoding='latin-1')

# load measurement data and crop according to the RoI
dint_red, dint_green, dint_ratio = splitImage(path=file_meas, RoI_op=RoI_op)

Image resolution: 1mm equals: 28.02px, i.e. 712dpi


In [6]:
# one example of one optode
inp = input('Which optode and which setting shall be visualized? ') # optode1, set2
kernel_str = input('Which kernel function should be applied? \n Choose between (1) Gauss or (2) Savitzky-Golay: \n > Your choise: ')

Which optode and which setting shall be visualized? optode1, set2
Which kernel function should be applied? 
 Choose between (1) Gauss or (2) Savitzky-Golay: 
 > Your choise: 1


### Prepare specific settings for data processing

In [7]:
# preparation lineprofile
surface = (12.9, 12.)   # optode 1, optode2
depth_lp = (-4, 4)      # depth for 2D line profile 
depth_op =  (-1.5, 1.)  # depth for optode excerpt
pos_lp = (7., 3.)       # position of the depth profile within the optode
ls_lw = [0]             # line with of the drawn depth profile

# .................................................................
# additional information for visualization
arg = dict({'curve lw': 1.5, 'vmin':-5, 'vmax': 130, 'lw': 0.0, 'offset ms': 0.2,
           'aspect': 4, 'cmap': plt.cm.inferno, 'vmin op': 0, 'vmax op': 100,
           'figsize': (12, 5), 'fontsize': 11, 'marker': ['o', 'd', 'x', '.'],
           'colors': ['slategrey', 'darkorange', 'forestgreen', 'k']})

Kernel specific settings

In [8]:
if kernel_str == str(1) or kernel_str == 'gauss':
    print('selected kernel function - Gauss filter')
    kernel='gauss'
    ls_kernel = [(1,1), (3,3), (5, 5), (7,7), (9,9), (11, 11), (13,13), (15, 15), (17,17),
                 (19,19), (21, 21), (25, 25), (31, 31), (35, 35), (41, 41), (45, 45), 
                 (51 ,51), (55, 55), (61, 61), (65, 65), (71, 71), (75, 75), (81, 81)]  
elif kernel_str == str(2) or kernel_str == 'Savitzky-Golay':
    print('selected kernel function - Savitzky-Golay filter')
    kernel='savgol'
    # polyorder, window
    ls_kernel = [(2,3), (2,5), (2,7), (2,9), (2,11), (2,13), (2,15), (2,17), (2,19), 
                 (2,21), (2,31), (2,41), (2, 51), (2,61), (2,71), (2,81),
                 (3,5), (3,7), (3,9), (3,11), (3,13), (3,15), (3,17), (3,19), (3,21), 
                 (3,31), (3,41), (3,51), (3,61), (3,71), (3,81)]
else:
    raise ValueError('Selection not valid. Choose either 1 (for Gauss) or 2 Savitzky-Golay')

selected kernel function - Gauss filter


Select, when blur filter should be applied

In [9]:
blur_str = input('When should the blur be applied? (1) single color channels, (2) ratiometric intensity, or (3) normalized intensity? \n > Your choise: ')

When should the blur be applied? (1) single color channels, (2) ratiometric intensity, or (3) normalized intensity? 
 > Your choise: 2


In [10]:
if blur_str == str(1):
    dint_ch1, dint_ch2 = dint_red, dint_green
    blur_pos='single'
elif blur_str == str(2):
    dint_ch1, dint_ch2 = dint_ratio, None
    blur_pos='ratio'
elif blur_str == str(3):
    dint_ch1, dint_ch2 = dint_ratio, None
    blur_pos='norm'
else:
    raise ValueError(' Selection not valid! \n Select either 1 (for single color channels), 2 (for ratiometric intensity), or 3 (for normalized intensity)')

### Image processing

In [28]:
# determine O2 concentration for the depth profile (1D) for different image blurs
dO2_lp = dict(map(lambda k: 
                  (k, O2_lineprofile_compare_v2(inp=inp, surface=surface, kernel=kernel, kshape=k, lp=pos_lp, 
                                                path_calib=file_calib, dint_ch1=dint_ch1, dint_ch2=dint_ch2, 
                                                px2mm=px2mm, ls_lw=ls_lw, blur_pos=blur_pos)), ls_kernel))
print('lineprofile done')

# determine O2 concentration within the whole image
dO2_optode = dict(map(lambda k: 
                      (k, O2blur_optode(kshape=k, inp=inp, path_calib=file_calib, px2mm=px2mm, kernel=kernel, 
                                        dint_ch1=dint_ch1, dint_ch2=dint_ch2, surface=surface, blur_pos=blur_pos,
                                        depth_min=depth_op[0], depth_max=depth_op[1])), ls_kernel))

lineprofile done


Visualize image blur along line profile

In [29]:
# prepare microsensor for joint visualization
df_ms = prepMS_plot(index_lp=dO2_lp[ls_kernel[0]]['vertical'][ls_lw[0]].index, dic_micro=dic_micro, 
                    offset=arg['offset ms'])

# plot lineprofile and excerpt of optode 
dimages = dict()
for k in ls_kernel:
    fig_lp = plotLP(kshape=k, dO2_lp=dO2_lp, dO2_optode=dO2_optode, df_ms=df_ms, arg=arg, 
                    header_ms=['Depth (mm)', 'Intensity'], depth_lp=depth_lp, s=inp.split(',')[1].strip(),
                    depth=df_ms['Depth (mm)'])
    dimages[k] = fig_lp 

/Users/au652733/Python/Project_NoiseResolution/functions_NoiseStudy.py:448: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig_lp = plt.figure(figsize=(arg['figsize']), dpi=100)


O$_2$ penetration depth

In [30]:
# set O2 level used for calculating the penetration depth in %air
threshold = 5.

# O2 penetration depth - returns a list (mean, min, max)
ydepth = penetration_depth(dO2_lp=dO2_lp, ls_kernel=ls_kernel, df_ms=df_ms, 
                           treshold=threshold)

# plotting penetration depth for different kernel sizes and functions
fig_Rpen = plot_penetrationDepth(depth=ydepth[0], ls_kernel=[round(l[1] / px2mm, 5) 
                                                             for l in ls_kernel], arg=arg)
fig_Rpen.axes[0].set_xlabel('kernel size [mm]')
plt.show()

### Save plots

In [17]:
now = datetime.datetime.now() 

# .....................................
name_DP_ = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_DepthProfile_'
name_DP_ += kernel + '-blur_' + blur_pos +'Intensity_' + '-'.join([i.strip() 
                                                                   for i in inp.split(',')])
for k in dimages.keys():
    name_DP = name_DP_ + '_kernel-order' + str(k[0]) +'-window-'+ str(k[1]) + '.'
    for t in type_plot:
        dimages[k].savefig(name_DP + t, dpi=300, transparent=False)

# .....................................
name_pen = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_PenetrationDepth_'
name_pen += kernel + '-blur_' + blur_pos + 'Intensity-' + '-'.join([i.strip()
                                                                    for i in inp.split(',')])
for t in type_plot:
    if fig_Rpen:
        fig_Rpen.savefig(name_pen + '.' + t, dpi=300, transparent=False)